# Part 1: Data Collection with Selenium

#### https://mersakarya.medium.com/selenium-tutorial-scraping-glassdoor-com-in-10-minutes-3d0915c6d905
#### Selenium Tutorial: Scraping Glassdoor.com in 10 Minutes by Omer Sakarya
#### https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb

In [1]:
# pip install selenium

In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [3]:
jobs = []
num_jobs = 32
closeSocial = False
keyword = 'data scientist'
path = "C:/Users/User/Documents/intro_data_science/chromedriver"
url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
slp_time=8

In [4]:
def closeSocialLogin(driver):
    try:
        driver.find_element_by_css_selector('[alt="Close"]').click() 
    except NoSuchElementException:
        pass

def closeSignUpPrompt(driver):
    try:
        driver.find_element_by_class_name("selected").click()
    except:
        pass

In [5]:
def initWebDdriver(webdriver):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 800)
    driver.get(url)
    print("webdriver initiated...")
    return driver

In [6]:
job_listing = "li.react-job-listing.job-search-key-nhtksm"

glassdoor_job_description = [
    ['company_name', '//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[1]'],
    ['location', '//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[3]' ],
    ['job_title', '//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[2]'],
    ['job_description', '//*[@id="JobDescriptionContainer"]'],
    ['salary_estimate', '//*[@id="JDCol"]/div/article/div/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]'],
    ['rating','//*[@id="employerStats"]/div[1]/div[1]'],
    ['size', '//*[@id="EmpBasicInfo"]/div[1]/div/div[1]/span[2]' ],
    ['founded', '//*[@id="EmpBasicInfo"]/div[1]/div/div[2]'],
    ['type_of_ownership', '//*[@id="EmpBasicInfo"]/div[1]/div/div[2]'],
    ['industry', '//*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span[2]'],
    ['sector','//*[@id="EmpBasicInfo"]/div[1]/div/div[5]'],
    ['revenue','//*[@id="EmpBasicInfo"]/div[1]/div/div[3]'],
]

In [7]:
def pullJobDescription(driver, glassdoor_xpaths):
    print("starting pull...")
    job_info={}
    for i in range(len(glassdoor_xpaths)):
        try:
            job_info[glassdoor_xpaths[i][0]] = driver.find_element_by_xpath(glassdoor_xpaths[i][1]).text
        except:
            job_info[glassdoor_xpaths[i][0]] = -1
    #print(job_info)
    return job_info

In [8]:
driver = initWebDdriver(webdriver)
    
while len(jobs) < num_jobs:  
    # wait for page to be loaded
    time.sleep(slp_time)
    closeSignUpPrompt(driver)        
    job_buttons = driver.find_elements_by_css_selector(job_listing)
        
    #for job_button in job_buttons:
    for i in range(5):
        print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
        if len(jobs) >= num_jobs:
            break 
        driver.execute_script("arguments[0].click();", job_buttons[i])
        time.sleep(1)
        closeSocialLogin(driver)
        time.sleep(1)
        job_info = pullJobDescription(driver, glassdoor_job_description)
        jobs.append(job_info)
        
    try:
        driver.find_element_by_xpath('.//li[@class="nextButton"]//span').click() #next page
        time.sleep(slp_time)
    except NoSuchElementException:
        print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
        break


<ipython-input-5-d6c1023234a6>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


webdriver initiated...


<ipython-input-4-22a32b831bc1>:9: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("selected").click()
<ipython-input-8-8b443581036d>:7: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  job_buttons = driver.find_elements_by_css_selector(job_listing)


Progress: 0/32


<ipython-input-4-22a32b831bc1>:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('[alt="Close"]').click()


starting pull...


<ipython-input-7-0e665d0f8a37>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  job_info[glassdoor_xpaths[i][0]] = driver.find_element_by_xpath(glassdoor_xpaths[i][1]).text


Progress: 1/32
starting pull...
Progress: 2/32
starting pull...
Progress: 3/32
starting pull...
Progress: 4/32
starting pull...
Scraping terminated before reaching target number of jobs. Needed 32, got 5.


<ipython-input-8-8b443581036d>:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('.//li[@class="nextButton"]//span').click() #next page


In [9]:
df = pd.DataFrame(jobs).reset_index()
df.to_csv('Uncleaned_DS_jobs.csv', index=False)

In [10]:
df[:10]

,index,company_name,location,job_title,job_description,salary_estimate,rating,size,founded,type_of_ownership,industry,sector,revenue
0,0,Amaze Systems Inc,Remote,Data Scientist,Job Description:\nPython\nMachine learning\nNe...,$57.50 /hr (est.),-1,Unknown,Type\nCompany - Public,Type\nCompany - Public,-1,-1,Revenue\nUnknown / Non-Applicable
1,1,Envision\n5.0,Remote,Data Scientist - Intermediate,What You Will Do:\nDesign and prototype models...,Visit Envision Website,5.0,Unknown,Type\nCompany - Private,Type\nCompany - Private,-1,-1,Revenue\nUnknown / Non-Applicable
2,2,INNOVATIVE BI SOLUTIONS INC,Remote,Data Scientist,Urgent!!\nJob Description\nJob Title: Data Sci...,$45.00 /hr (est.),-1,-1,-1,-1,-1,-1,-1
3,3,Apollo Neuroscience,Remote,Data Scientist,Apollo Neuroscience is pioneering new ways to ...,Visit Apollo Neuroscience Website,-1,1 to 50 Employees,Type\nCompany - Private,Type\nCompany - Private,-1,-1,Revenue\nUnknown / Non-Applicable
4,4,Highland Principals,Remote,Data Scientist,This is a new role within the R&D organization...,$85.00 /hr (est.),-1,Unknown,Type\nCompany - Public,Type\nCompany - Public,-1,-1,Revenue\nUnknown / Non-Applicable
